# Creating Types

## Outline
* Type synonyms
	* What
	* Why
	* Type Parameters 
* Defining new types
	* data
	* Value Parameters
	* Type Parameters
	* Record syntax
	* deriving
	* newType

## Type Synonyms

Early on, when learning about `Strings` in Haskell, you found out that `String` is syntactic sugar of `[Char]`. They're the same but easier to read.

That's because `String` is a type synonym for `[Char]`.

This means that `String` and `[Char]` are *equivalent* and you can use them *interchangeably*.

### How to define Type Synonyms

```haskell
type String = [Char]
```

* The `type` keyword defines a type synonym.
* You can name the type synonym however you want, as long as it starts with a capital letter.

**When you define a type synonym, you're not creating a new type! You're telling Haskell that an existing type can be referred with a different name (a synonym)!**

### Why use Type Synonyms

Why would you add more complexity without adding more functionality?

Because type synonyms allow us to convey more information! Let's see an example.

Imagine you started working with a library that allows you to create monetary transactions.

You want to create a new transaction, so you take a look at the type signature of the function that you need to use:

```haskell
generateTx :: String -> String -> Int -> String 
```

Not an extremely useful signature. You could infer that the Int is the value to transfer, but what are those Strings? And what does that String that it returns contain?

Now, compare this type signature:

```haskell
generateTx :: String -> String -> Int -> String 
```

Whith this one:

```haskell
generateTx :: Address -> Address -> Value -> Id
```

Clearly, the second signature transmits the context way better!
The first two parameters are addresses, and the third one is the value of the transaction. Finally, we know that it returns the id of the transaction. 

All that just from the type signature. The difference? Just a few type synonyms.

And what did we do to improve the context so much? Let's see:

Let's recreate the function called `generateTx` that will take the addresses and value of a transaction and generates a id for it:

In [ ]:
generateTxHash :: String -> String -> Int -> String 
generateTxHash from to value = from ++ to ++ show value

Now, we just need to add some type synonyms:

In [ ]:
type Address = String
type Value = Int
type Id = String

generateTx :: Address -> Address -> Value -> Id
generateTx from to value = from ++ to ++ show value

Finally, if you want to check what does the `Address`, `Value`, or `Id` types are, you can open GHCi, load the file and check its info:

In [ ]:
:i Address 
:i Value
:i Id

Another example:

In [ ]:
type Name = String
type Address = (String, Int)
type Person = (Name, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Person
:t bob
:t fst bob

But what if we need a more flexible type?

In that case, we can use parameterized type synonyms!

### Paremeterizing Type Synonyms

Imagine that after using our recently created Person type synonym, we find out that we also have to identify companies.

We could do something like:

In [ ]:
type Name = String
type Address = (String, Int)
type Person = (Name , Address)

type CompanyId = Int
type Company = (CompanyId, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Person
io = ("Input|Output", ("Cardano St.", 999)) :: Company

But if we do that, we repeat a lot of code and each time we need to add a new entity. A better aproach would be to define a parametric type synonym.

We add a parameter to the type synonym definition—same as with functions—and we can create different types for different situations!:

In [ ]:
type Name = String
type Address = (String, Int)
type CompanyId = Int

type Entity a = (a, Address)

bob = ("Bob Smith", ("Main St.", 555)) :: Entity Name 
io = (584264, ("Cardano St.", 999)) :: Entity CompanyId

Notice that `Entity Name` and `Entity CompanyId` are completely different types! The same as `Int` and `Bool` are different.

Having type synonyms it's cool and all, but they are just different names for the same thing. What if we need to create a brand new type? `data` to the rescue!

## Defining new types

### `data` keyword

We can create new types using the data keyword like this:

In [ ]:
data PaymentMethod = Cash | Card | Cryptocurrency

data Color = Red | Blue | Green

data Bool = True | False

* The part before the equal sign is our new type name.

* The part after the equal sign are value constructors. Value constructors specify the different values that the type can have.

* The `|` is read as "or". So we can read the firs type as:

		The type `PaymentMethod` can have a value of `Cash`, `Card`, or `Cryptocurrency`.

<div class="alert alert-block alert-warning">
<b>Warning:</b> The type name and the value constructors must start with an uppercase letter!
</div>

**And now, how can we use a new type?**

**By using its values!** For example, lets add a payment method to our entities:

In [ ]:
type Name = String
type Address = (String, Int)
type CompanyId = Int

data PaymentMethod = Cash | Card | Cryptocurrency

type Entity a = (a, Address, PaymentMethod)

bob = ("Bob Smith", ("Main St.", 555), Cash) :: Entity Name 
io = (584264, ("Cardano St.", 999), Cryptocurrency) :: Entity CompanyId

Now we can use the `PaymentMethod` type however we want:

In [ ]:
howItPays :: Entity a -> String
howItPays (_, _, Cash) = "Pays in cash" -- Pattern matching Entity and ignoring everything except the PaymentMethod
howItPays (_, _, Card) = "Pays with card"
howItPays (_, _, Cryptocurrency) = "Pays with cryptocurrency"

howItPays bob
howItPays io

But this generates a question. What can we do if we need more than a few values?

What if, for example, I want a type to represent a shape that could be any circle or any rectangle?

We can start by defining something like:

In [ ]:
data Shape = Circle | Rectangle

But the thing is, this isn't of much use. I want to be able to do stuff with these values, like calculating perimeters and areas.

No problem at all! We can just add some parameters to the values!

### Value Parameters

Let's add some parameters:
* To define a circle, we need its radius. So just one numeric value.
* To define a rectangle, we need the length of its two sides. So two numeric values.

Let's translate those requirements to code!:

In [ ]:

data Shape = Circle Float | Rectangle Float Float

:t Circle
:t Rectangle

As you can see, if we check the type of the `Circle` constructor, we see that it's actually a function!!

A function that takes a value of type `Float` and returns a value of type `Shape`! So, to obtain our circle of type `Shape`, all we have to do is pass its radius:

In [ ]:
smallCircle = Circle 10

hugeCircle = Circle 1000

Same for `Rectangle` values. 

`Rectangle` is a function that takes two values of type `Float` and returns a value of type `Shape`. So, to obtain a rectangle of type `Shape`, all we have to do is pass the lengths of its sides:

In [ ]:
rect1 = Rectangle 10 5

rect2 = Rectangle 256 128

<div class="alert alert-block alert-warning">
<b>Warning:</b>
<p>
If you play around with creating new types and you get the error:
 </p>

```
Error: No instance for (Show Shape) arising from a use of print.
```

<p>
Just add <code>deriving (Show)</code> at the end of the type definition like this:
</p>

```haskell
data Shape = Circle Float | Rectangle Float Float deriving (Show)
```

<p>
We'll explain what is this at the end of this lesson.
</p>
</div>

We created our new type, and created some values. Now let's use them!

We can define a function that calculates the area of any value of type `Shape` like this:

In [ ]:
area :: Shape -> Float
area (Circle r) = pi * r^2        -- We pattern match on value constructors and bind its fields to names.
area (Rectangle l1 l2) = l1 * l2


area smallCircle
area rect2

Now we're talking! We just created a useful type!

But that's not all! We can also create parametric types!

### Parameterizing new types

### Record Syntax

### Deriving

### `newType` keyword